In [57]:
import tabula
import pandas as pd 
import datetime
import boto3

In [39]:
def to_date(x):
    try:
        x = list(map(int, x.split('-')))
        date = datetime.date(x[0],x[1],x[2])
    except Exception as e:
        print(e) 
        date = None

    return date


def extract_pdf(pdf_url):
    df = pd.concat(tabula.read_pdf(pdf_url, pages = 'all'))
    df['date_payment_confirmed'] = df['date_payment_confirmed'].apply(to_date)
    df['expiry_date'] = pd.to_datetime(df['expiry_date'],format = '%m/%y',errors= 'coerce').dt.date
    return df
    

In [40]:
pdf_url = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'

card_data = extract_pdf(pdf_url)

invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'GTC9KBWJO9'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'DJIXF1AFAZ'
invalid literal for int() with base 10: 'December 2021 17'
invalid literal for int() with base 10: '2005 July 01'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'H2PCQP4W50'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'XTD27ANR5Q'
invalid literal for int() with base 10: '7VGB4DA1WI'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'RLQYRRYHPU'
invalid literal for int() with base 10: 'December 2000 01'
invalid literal for int() with base 10: '2008 May 11'
invalid literal for int() with b

In [13]:
card_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15309 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15284 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15276 non-null  object
dtypes: object(4)
memory usage: 598.0+ KB


In [14]:
card_data

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,2026-09-01,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,2023-10-01,American Express,2001-06-18
2,3529023891650490,2023-06-01,JCB 16 digit,2000-12-26
3,213142929492281,2027-09-01,JCB 15 digit,2011-02-12
4,502067329974,2025-10-01,Maestro,1997-03-13
...,...,...,...,...
14,180036921556789,2028-12-01,JCB 15 digit,1997-06-06
15,180018030448512,2024-11-01,JCB 15 digit,2004-06-16
16,3569953313547220,2024-04-01,JCB 16 digit,2020-02-05
17,4444521712606810,2027-06-01,VISA 16 digit,2008-06-16


In [15]:
card_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15309 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15284 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15276 non-null  object
dtypes: object(4)
memory usage: 598.0+ KB


In [16]:
card_data.isna().sum()

card_number                0
expiry_date               25
card_provider              0
date_payment_confirmed    33
dtype: int64

In [17]:
card_data['card_number'].astype(int)

ValueError: invalid literal for int() with base 10: '?4971858637664481'

In [25]:
def clean_questionmark(x):
    try:
        if x == "NULL":
            return None
        x = str(x).replace('?',"")
        return x 
    except Exception as e:
        print(e) 



In [28]:
card_data['card_number'] = card_data['card_number'].apply(clean_questionmark)

In [29]:
card_data.isna().sum()

card_number               11
expiry_date               25
card_provider              0
date_payment_confirmed    33
dtype: int64

In [35]:
(card_data['card_provider'] == 'NULL').sum()

11

In [34]:
card_data.isna().sum()

card_number               11
expiry_date               25
card_provider              0
date_payment_confirmed    33
dtype: int64

In [36]:
card_data.loc[card_data['card_provider'] == 'NULL', 'card_provider'] = None

In [37]:
card_data.isna().sum()

card_number               11
expiry_date               25
card_provider             11
date_payment_confirmed    33
dtype: int64

In [38]:
def clean_card_data(card_data):
    card_data['card_number'] = card_data['card_number'].apply(clean_questionmark)
    card_data.loc[card_data['card_provider'] == 'NULL', 'card_provider'] = None
    return card_data.dropna()

    
    


In [41]:
new_data = clean_card_data(card_data)

In [42]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15276 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15276 non-null  object
 1   expiry_date             15276 non-null  object
 2   card_provider           15276 non-null  object
 3   date_payment_confirmed  15276 non-null  object
dtypes: object(4)
memory usage: 596.7+ KB


In [43]:
new_data.isna().sum()

card_number               0
expiry_date               0
card_provider             0
date_payment_confirmed    0
dtype: int64

In [45]:
(new_data['card_number']== 'NULL').sum()

0

In [1]:
from data_extraction import DataExtractor


    

In [2]:
extractor = DataExtractor("www.test.com")

In [4]:
df_stores = extractor.retrieve_stores_data('https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}')

In [5]:
df_stores.info()


<class 'pandas.core.frame.DataFrame'>
Index: 451 entries, 0 to 450
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   address        451 non-null    object
 1   longitude      451 non-null    object
 2   lat            11 non-null     object
 3   locality       451 non-null    object
 4   store_code     451 non-null    object
 5   staff_numbers  451 non-null    object
 6   opening_date   451 non-null    object
 7   store_type     451 non-null    object
 8   latitude       450 non-null    object
 9   country_code   451 non-null    object
 10  continent      451 non-null    object
dtypes: object(11)
memory usage: 42.3+ KB


In [12]:
df_stores.head()

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
index,,,,,,,,,,
0,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,"Flat 72W Sally isle East Deantown E7B 8EB, Hig...",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5 50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel South Lydia WC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,"Studio 6 Stephen landing South Simon B77 2WA, ...",53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe


In [21]:
df_stores.drop('lat', inplace = True, axis = 1)
df_stores.drop(447, inplace= True)

In [10]:
df_stores['address'].apply(lambda x: x.replace('\n', ' '))

index
0                                                    N/A
1      Flat 72W Sally isle East Deantown E7B 8EB, Hig...
2             Heckerstraße 4/5 50491 Säckingen, Landshut
3        5 Harrison tunnel South Lydia WC9 2BE, Westbury
4      Studio 6 Stephen landing South Simon B77 2WA, ...
                             ...                        
446       Täschestraße 25 39039 Nördlingen, Kirchlengern
447                                           K0ODETRLS3
448    Studio 8 Moss mall West Linda M0E 6XR, High Wy...
449                  Baumplatz 6 80114 Kötzting, Bretten
450    Gotthilf-Rose-Straße 7/3 45457 Feuchtwangen, B...
Name: address, Length: 451, dtype: object

In [11]:
df_stores['address'] = df_stores['address'].apply(lambda x: x.replace('\n', ' '))

In [17]:
df_stores.loc[(df_stores['longitude'] == 'N/A'),'longitude'] = None

In [27]:
def convert_to_num(x, type_1):
    try:
        return type_1(x) 
    except: 
        return None 
    


In [30]:
df_stores['longitude'] = df_stores['longitude'].apply(convert_to_num, type_1 = float)
df_stores['latitude'] = df_stores['latitude'].apply(convert_to_num, type_1 = float)
df_stores['staff_numbers'] = df_stores['staff_numbers'].apply(convert_to_num, type_1 = int)


In [33]:
df_stores['opening_date'].apply(convert_to_num, type_1 = datetime.date)

index
0      None
1      None
2      None
3      None
4      None
       ... 
445    None
446    None
448    None
449    None
450    None
Name: opening_date, Length: 450, dtype: object

In [35]:
def date_format(x):
    try: 
        if '-' in x:
            x = list(map(int, x.split('-')))
            return datetime.date(x[0],x[1],x[2])
        else:
            return datetime.datetime.strptime(x, '%B %Y %d').date()
    except: 
        return None
        

        

In [36]:
df_stores['opening_date'].apply(date_format)

index
0      2010-06-12
1      1996-10-25
2      2013-04-12
3      2014-01-02
4      2019-09-09
          ...    
445    2008-06-08
446    2005-05-12
448    1998-05-14
449    2020-10-17
450    2001-05-12
Name: opening_date, Length: 450, dtype: object

In [37]:
df_stores['opening_date']= df_stores['opening_date'].apply(date_format)

In [38]:
df_stores.head()

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
index,,,,,,,,,,
0,N/A,NaN,N/A,WEB-1388012W,325.0,2010-06-12,Web Portal,NaN,GB,Europe
1,"Flat 72W Sally isle East Deantown E7B 8EB, Hig...",51.62907,High Wycombe,HI-9B97EE4E,34.0,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5 50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92.0,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel South Lydia WC9 2BE, Westbury",51.26000,Westbury,WE-1DE82CEE,69.0,2014-01-02,Super Store,-2.18750,GB,Europe
4,"Studio 6 Stephen landing South Simon B77 2WA, ...",53.02330,Belper,BE-18074576,35.0,2019-09-09,Local,-1.48119,GB,Europe


In [39]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 450 entries, 0 to 450
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   address        450 non-null    object 
 1   longitude      440 non-null    float64
 2   locality       450 non-null    object 
 3   store_code     450 non-null    object 
 4   staff_numbers  436 non-null    float64
 5   opening_date   435 non-null    object 
 6   store_type     450 non-null    object 
 7   latitude       440 non-null    float64
 8   country_code   450 non-null    object 
 9   continent      450 non-null    object 
dtypes: float64(3), object(7)
memory usage: 38.7+ KB


In [42]:
for col in df_stores.columns: 
    df_stores[col] = df_stores[col].replace({'N/A': None})


In [44]:
df_stores.isna().sum()

address           1
longitude        10
locality          1
store_code        0
staff_numbers    14
opening_date     15
store_type        0
latitude         10
country_code      0
continent         0
dtype: int64

In [46]:
df_stores.dropna(inplace = True)

In [47]:
df_stores.isna().sum()

address          0
longitude        0
locality         0
store_code       0
staff_numbers    0
opening_date     0
store_type       0
latitude         0
country_code     0
continent        0
dtype: int64

In [48]:
df_stores.head()

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
index,,,,,,,,,,
1,"Flat 72W Sally isle East Deantown E7B 8EB, Hig...",51.62907,High Wycombe,HI-9B97EE4E,34.0,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5 50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92.0,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel South Lydia WC9 2BE, Westbury",51.26000,Westbury,WE-1DE82CEE,69.0,2014-01-02,Super Store,-2.18750,GB,Europe
4,"Studio 6 Stephen landing South Simon B77 2WA, ...",53.02330,Belper,BE-18074576,35.0,2019-09-09,Local,-1.48119,GB,Europe
5,Flat 92u Christian harbors Port Charlotte N57 ...,53.38333,Gainsborough,GA-CAD01AC2,36.0,1995-05-15,Local,-0.76667,GB,Europe


In [53]:
for col in df_stores.columns: 
    print(type(df_stores[col].iloc[0]))

<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'datetime.date'>
<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>


In [55]:
def get_type(col_type):
    if 'str' in str(col_type):
        return 'VARCHAR(225)'
    elif 'float' in str(col_type):
        return 'FLOAT'
    elif 'int' in str(col_type): 
        return 'INT'
    elif 'date' in str(col_type):
        return 'DATE'
    else: 
        return 'VARCHAR(225)'
    
    

In [56]:
for col in df_stores.columns: 
    print(get_type(type(df_stores[col].iloc[0])))

VARCHAR(225)
FLOAT
VARCHAR(225)
VARCHAR(225)
FLOAT
DATE
VARCHAR(225)
FLOAT
VARCHAR(225)
VARCHAR(225)


In [59]:
s3 = boto3.client('s3', region_name = 'eu-west-2', aws_access_key_id = 'aenvoiwmvomwiv', aws_secret_access_key = 'ianermgionw78')
